# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [38]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
#CSV into DF
path = "../output_data/cities.csv"
cities_info = pd.read_csv(path)

#Remove blank data rows
cities_info = cities_info.dropna()
len(cities_info)


551

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
#config google maps
gmaps.configure(api_key=g_key)

#Get lat/lng locations vs humidity
location = cities_info[["Latitude", "Longitude"]]
humidity  = cities_info["Humidity"]

In [41]:
#heatmap layer to map (use func)
figure = gmaps.figure()

heat_layer = gmaps.heatmap_layer( location,
                                  weights = humidity, 
                                  dissipating = False,
                                  max_intensity = 100,
                                  point_radius  = 1)
                                
figure.add_layer(heat_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [51]:
#narrow down cities ~ weather conditions
cities_weather = cities_info.loc[ (   (cities_info["Temperature"] > 70) 
                               & (cities_info["Temperature"] < 80)) 
                               & (cities_info["Wind speed"] < 10 ) 
                               & (cities_info["Cloudiness"] == 0),
                                [ "City","Country", "Latitude", "Longitude" ]].reset_index(drop=True) 
len(cities_weather)
cities_weather

,City,Country,Latitude,Longitude
0,Agía Galíni,GR,35.1000,24.6833
1,Redondela,ES,42.2834,-8.6096
2,Fairbanks,US,64.8378,-147.7164
3,Broome,US,42.2506,-75.8330
4,Kiryat Gat,IL,31.6100,34.7642
5,São Miguel do Araguaia,BR,-13.2750,-50.1628
6,Hailar,CN,49.2000,119.7000
7,Guidonia,IT,42.0167,12.7500
8,Westport,US,41.1415,-73.3579
9,Capoterra,IT,39.1763,8.9718


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
#step 1 --> df w/ new col for hotel names
hotel_info = cities_weather

#step 2 --> hotel name col.
hotel_info["Hotel Name"] = ""

#step 3 --> params for hotels 5000m
params = {"keyword": "hotel",
          "radius": 5000,
          "type": "lodging",
          "key": g_key}

#step 4 --> url to find google places
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [59]:
#step 5 --> loop through hotels for cities in df
for index, row in hotel_info.iterrows():
    
    #set up params
    latitude  = row["Latitude"]
    longitude = row["Longitude"]
    params["location"] = f"{latitude}, {longitude}"
    
    #url
    city = row["City"]
    print(f"Obtaining Hotel Data for set {index}: city {city}.")
    response = requests.get(hotel_url, params = params, verify = False)
    response = response.json()
    outcome = response['results']
    
    #try|except for troubleshooting
    try:
        print(f"Closest hotel in {city} is {outcome[0]['name']}.")
        #first hotel stored to df
        hotel_info.loc[index, 'Hotel Name'] = outcome[0]['name']
    
    except (KeyError, IndexError):
        print("Missing/ no hotel... skipping.")    
        print("------------------------------")

hotel_info

Obtaining Hotel Data for set 0: city Agía Galíni.


C:\Users\COOPERKK\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Closest hotel in Agía Galíni is Palazzo Greco.
Obtaining Hotel Data for set 1: city Redondela.


C:\Users\COOPERKK\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Closest hotel in Redondela is Hotel Antolín.
Obtaining Hotel Data for set 2: city Fairbanks.


...

C:\Users\COOPERKK\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Closest hotel in Kununurra is The Kimberley Grande Resort.
Obtaining Hotel Data for set 20: city Barcelos.


C:\Users\COOPERKK\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Closest hotel in Barcelos is Art'otel Barcelos.


,City,Country,Latitude,Longitude,Hotel Name
0,Agía Galíni,GR,35.1000,24.6833,Palazzo Greco
1,Redondela,ES,42.2834,-8.6096,Hotel Antolín
2,Fairbanks,US,64.8378,-147.7164,SpringHill Suites by Marriott Fairbanks
3,Broome,US,42.2506,-75.8330,
4,Kiryat Gat,IL,31.6100,34.7642,Desert Gat
5,São Miguel do Araguaia,BR,-13.2750,-50.1628,Hotel Executivo Palace
6,Hailar,CN,49.2000,119.7000,Hulunbeir Hotel
7,Guidonia,IT,42.0167,12.7500,Villa Verde
8,Westport,US,41.1415,-73.3579,Norwalk Inn
9,Capoterra,IT,39.1763,8.9718,Hotel Santa Gilla


In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#my df was called hotel_info, so switched df names around
hotel_df = [info_box_template.format(**row) for index, row in hotel_info.iterrows()]
locations = hotel_info[["Latitude", "Longitude"]]

In [65]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations)

hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=2, info_box_content=hotel_df)

#plot
fig = gmaps.figure()
fig.add_layer(marker)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))